In [1]:
import pandas as pd
import pycountry
import warnings
import random
warnings.filterwarnings("ignore")
import numpy as np

In [2]:
iso2 = {country.alpha_2: country.name.lower() for country in pycountry.countries}
iso3 = {country.alpha_3 : country.name for country in pycountry.countries}
iso32 = {country.alpha_3 : country.alpha_2 for country in pycountry.countries}
iso2_inverse = {country.name.upper():country.alpha_2 for country in pycountry.countries}
iso3_inverse = {country.name.upper(): country.alpha_3 for country in pycountry.countries}

# Population number
Population number from the UN

In [3]:
population = pd.read_csv('country_data/population.csv', error_bad_lines=False)[['Country Code', 'Country Name', '2019']]

# Number of households
number of households are coming from: https://www.prb.org/international/indicator/hh-size-av/map/country/

In [4]:
def get_iso3(country): 
    if country == "Cote d'Ivoire":
        country = "Côte d'Ivoire"
    elif country == "Bolivia":
        country = 'Bolivia, Plurinational State of'
    elif country == "Dem. Republic of the Congo":
        country = 'Congo, The Democratic Republic of the'
    elif country == 'Iran':
        country = 'Iran, Islamic Republic of'
    elif country == "Lao People's Dem. Republic":
        country = "Lao People's Democratic Republic"
    elif country == 'Republic of Moldova':
        country = 'Moldova, Republic of'
    elif country == 'Reunion':
        country = 'Réunion'
    elif country == 'Saint-Barthelemy':
        country = 'Saint Barthélemy'
    elif country == 'Saint-Martin (French part)':
        country = 'Saint Martin (French part)'
    elif country == 'State of Palestine':
        country = 'Palestine, State of'
    elif country == 'Tanzania':
        country = 'Tanzania, United Republic of'
    elif country == 'Venezuela':
        country = 'Venezuela, Bolivarian Republic of'
    elif country == 'Vietnam':
        country = 'Viet Nam'
    elif country == 'Swaziland':
        country = 'Eswatini'
    try:
        return iso3_inverse[country.upper()]
    except:
        print('ISO-3 mapping wrong for:')
        print(country)

In [5]:
number_of_households = pd.read_csv('country_data/number_of_households.csv', error_bad_lines=True)
number_of_households['ISO3'] = number_of_households.Country.apply(get_iso3)
number_of_households = number_of_households[['ISO3', 'noh']]

In [6]:
number_of_households

,ISO3,noh
0,AFG,8.036739
1,ALB,3.299611
2,AGO,4.815657
3,ARG,3.258590
4,ARM,3.542085
...,...,...
155,VEN,4.331362
156,VNM,3.783099
157,YEM,6.672681
158,ZMB,5.130789


In [6]:
data = number_of_households.merge(population, left_on='ISO3', right_on='Country Code')

In [7]:
def iso3_country_name(inp):
    return iso3[inp]

def iso3_2(inp):
    return iso32[inp]

# Languages

In [8]:
import babel
from babel import languages
language_iso_name = {}
for language in pycountry.languages:
    try:
        language_iso_name.update({language.alpha_2: language.name, language.alpha_3: language.name})
    except:
        language_iso_name.update({language.alpha_3: language.name})

In [9]:
def get_languages(iso3):
    langs = babel.languages.get_official_languages(iso32[iso3])
    lan = []
    for language in langs:
        try:
            lan.append(language_iso_name[language])
        except:
            try:
                lan.append(language_iso_name[language[0:2]])
            except:
                print(language)
    return ', '.join(lan)

def translation_yes_no(iso3):
    langs = babel.languages.get_official_languages(iso32[iso3])
    if 'en' in langs:
        return None
    else:
        return 'Yes'

In [10]:
data['languages'] = data['ISO3'].apply(get_languages)

In [11]:
data.to_csv('map_data.csv', index=False)